In [ ]:
import pandas as pd
import numpy as np
import glob

# 파일 불러오기

In [2]:
def load_data(data_type):
    dfs = {}
    base_path = f"open/{data_type}"
    categories = {
        "customer": "1",
        "credit": "2",
        "sales": "3",
        "billing": "4",
        "balance": "5",
        "channel": "6",
        "marketing": "7",
        "performance": "8"
    }
    for name, prefix in categories.items():
        path =  f"{base_path}/{prefix}.*/*.parquet"
        files = sorted(glob.glob(path))
        if not files:
            print(f"No parquet files found in {path}")
            continue
        dfs[name] = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)
        print(f"{name} Loaded {dfs[name].shape} parquet files")
    return dfs

In [3]:
train_df = load_data("train")

customer Loaded (2400000, 78) parquet files
credit Loaded (2400000, 42) parquet files
sales Loaded (2400000, 406) parquet files
billing Loaded (2400000, 46) parquet files
balance Loaded (2400000, 82) parquet files
channel Loaded (2400000, 105) parquet files
marketing Loaded (2400000, 64) parquet files
performance Loaded (2400000, 49) parquet files


In [4]:
test_df = load_data("test")

customer Loaded (600000, 77) parquet files
credit Loaded (600000, 42) parquet files
sales Loaded (600000, 406) parquet files
billing Loaded (600000, 46) parquet files
balance Loaded (600000, 82) parquet files
channel Loaded (600000, 105) parquet files
marketing Loaded (600000, 64) parquet files
performance Loaded (600000, 49) parquet files


In [5]:
customer_train_df = train_df["customer"]
credit_train_df   = train_df["credit"]
sales_train_df    = train_df["sales"]
billing_train_df  = train_df["billing"]
balance_train_df  = train_df["balance"]
channel_train_df  = train_df["channel"]
marketing_train_df= train_df["marketing"]
performance_train_df = train_df["performance"]

In [6]:
customer_test_df = test_df["customer"]
credit_test_df   = test_df["credit"]
sales_test_df    = test_df["sales"]
billing_test_df  = test_df["billing"]
balance_test_df  = test_df["balance"]
channel_test_df  = test_df["channel"]
marketing_test_df= test_df["marketing"]
performance_test_df = test_df["performance"]

In [7]:
def check_memory_usage(df):
    memory_usage = df.memory_usage(deep=True).sum() / 1024**2  # MB 단위로 변환
    print(f"메모리 사용량: {memory_usage:.2f} MB")
    return memory_usage

In [8]:
print("메모리 사용량 확인:")
check_memory_usage(customer_train_df)

메모리 사용량 확인:
메모리 사용량: 3751.89 MB


3751.8916912078857

In [9]:
import gc

In [10]:
import re

# 전처리 함수들
1. '개, 대, 회, 이상, 회이상, 개이상, 대이상'로 끝나는 문자열에서 숫자만 추출
2. '', 'none', 'null', 'nan', 'NaN' 이 문자열로 들어왔을 때 처리, 10101, -999999, -99, 999, 99999999 값 0 처리
3. unique 값이 1개인 컬럼 drop
4. IQR 방식으로 이상치 탐색 후 NaN 으로 대치 (날짜 형태의 컬럼은 0으로 처리)
5. 날짜형 데이터의 추가적인 결측치가 있을 경우 0처리
6. 각 컬럼별 결측치의 비율에 따라 평균값, 중앙값, 최빈값, drop등으로 처리

## 문자열 -> 수치형으로 파싱

'개, 대, 회, 이상, 회이상, 개이상, 대이상'로 끝나는 문자열에서 숫자만 추출


In [11]:
def clean_units_from_dataframe(df):
    # 데이터프레임 복사본 생성
    df_cleaned = df.copy()
    object_columns = df_cleaned.select_dtypes(include=['object']).columns
    # 각 object 컬럼 처리
    for col in object_columns:
        # 문자열인 경우에만 처리
        mask = df_cleaned[col].apply(lambda x: isinstance(x, str))

        if mask.any():  # 문자열이 하나 이상 있는 경우만 처리
            unit_pattern = r'\d+\s*(개|대|회|이상|회이상|개이상|대이상|회 이상|일 이상)$'
            # re.search를 직접 적용하여 경고 피하기
            unit_mask = df_cleaned.loc[mask, col].apply(lambda x: bool(re.search(unit_pattern, x)))

            # 단위가 있는 값만 처리
            if unit_mask.any():
                # 숫자 추출하기 위한 패턴
                number_pattern = r'(\d+)\s*(개|대|회|이상|회이상|개이상|대이상|회 이상|일 이상)$'
                # 숫자 부분만 추출
                extracted_numbers = df_cleaned.loc[mask, col].str.extract(number_pattern)[0]
                # 추출된 숫자를 새로운 값으로 사용 (숫자로 변환)
                df_cleaned.loc[mask & unit_mask, col] = pd.to_numeric(
                    extracted_numbers.loc[unit_mask], errors='coerce')
    return df_cleaned

In [12]:
customer_train_df = clean_units_from_dataframe(customer_train_df)
credit_train_df   = clean_units_from_dataframe(credit_train_df)
sales_train_df    = clean_units_from_dataframe(sales_train_df)
billing_train_df  = clean_units_from_dataframe(billing_train_df)
balance_train_df  = clean_units_from_dataframe(balance_train_df)
channel_train_df  = clean_units_from_dataframe(channel_train_df)
marketing_train_df= clean_units_from_dataframe(marketing_train_df)
performance_train_df = clean_units_from_dataframe(performance_train_df)

In [13]:
customer_test_df     = clean_units_from_dataframe(customer_test_df)
credit_test_df       = clean_units_from_dataframe(credit_test_df)
sales_test_df        = clean_units_from_dataframe(sales_test_df)
billing_test_df      = clean_units_from_dataframe(billing_test_df)
balance_test_df      = clean_units_from_dataframe(balance_test_df)
channel_test_df      = clean_units_from_dataframe(channel_test_df)
marketing_test_df    = clean_units_from_dataframe(marketing_test_df)
performance_test_df  = clean_units_from_dataframe(performance_test_df)

In [14]:
marketing_test_df.head()

,기준년월,ID,컨택건수_카드론_TM_B0M,컨택건수_리볼빙_TM_B0M,컨택건수_CA_TM_B0M,컨택건수_이용유도_TM_B0M,컨택건수_신용발급_TM_B0M,컨택건수_부대서비스_TM_B0M,컨택건수_포인트소진_TM_B0M,컨택건수_보험_TM_B0M,...,컨택건수_카드론_당사앱_R6M,컨택건수_CA_당사앱_R6M,컨택건수_리볼빙_당사앱_R6M,컨택건수_이용유도_당사앱_R6M,컨택건수_채권_B0M,컨택건수_FDS_B0M,컨택건수_채권_R6M,컨택건수_FDS_R6M,캠페인접촉건수_R12M,캠페인접촉일수_R12M
0,201807,TEST_00000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,201807,TEST_00001,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,5
2,201807,TEST_00002,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,10
3,201807,TEST_00003,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15,15
4,201807,TEST_00004,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


## 결측치 수기 처리

'', 'none', 'null', 'nan', 'NaN' 이 문자열로 들어왔을 때 처리, 10101, -999999, -99, 999, 99999999 값 0 처리

In [15]:
def is_special_null(df):
    df_copy = df.copy()

    # 문자열 컬럼 처리 (벡터화된 방식)
    for col in df_copy.select_dtypes(include=['object']).columns:
        # 문자열 컬럼에서 빈 문자열, 'none', 'null', 'nan' 찾기
        mask = df_copy[col].astype(str).str.lower().isin(['', 'none', 'null', 'nan', 'NaN'])
        df_copy.loc[mask, col] = np.nan  # NaN으로 변경

    # 숫자 컬럼 처리
    for col in df_copy.select_dtypes(include=['int64', 'float64']).columns:
        # 특정 숫자값 찾기
        mask = df_copy[col].isin([10101, -999999, -99, 999, 99999999])
        df_copy.loc[mask, col] = np.nan  # NaN으로 변경
    return df_copy

In [16]:
customer_train_df = is_special_null(customer_train_df)
credit_train_df   = is_special_null(credit_train_df)
sales_train_df    = is_special_null(sales_train_df)
billing_train_df  = is_special_null(billing_train_df)
balance_train_df  = is_special_null(balance_train_df)
channel_train_df  = is_special_null(channel_train_df)
marketing_train_df= is_special_null(marketing_train_df)
performance_train_df = is_special_null(performance_train_df)

In [17]:
customer_test_df     = is_special_null(customer_test_df)
credit_test_df       = is_special_null(credit_test_df)
sales_test_df        = is_special_null(sales_test_df)
billing_test_df      = is_special_null(billing_test_df)
balance_test_df      = is_special_null(balance_test_df)
channel_test_df      = is_special_null(channel_test_df)
marketing_test_df    = is_special_null(marketing_test_df)
performance_test_df  = is_special_null(performance_test_df)

In [18]:
marketing_test_df.head()

,기준년월,ID,컨택건수_카드론_TM_B0M,컨택건수_리볼빙_TM_B0M,컨택건수_CA_TM_B0M,컨택건수_이용유도_TM_B0M,컨택건수_신용발급_TM_B0M,컨택건수_부대서비스_TM_B0M,컨택건수_포인트소진_TM_B0M,컨택건수_보험_TM_B0M,...,컨택건수_카드론_당사앱_R6M,컨택건수_CA_당사앱_R6M,컨택건수_리볼빙_당사앱_R6M,컨택건수_이용유도_당사앱_R6M,컨택건수_채권_B0M,컨택건수_FDS_B0M,컨택건수_채권_R6M,컨택건수_FDS_R6M,캠페인접촉건수_R12M,캠페인접촉일수_R12M
0,201807.0,TEST_00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,201807.0,TEST_00001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5
2,201807.0,TEST_00002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,10
3,201807.0,TEST_00003,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,15
4,201807.0,TEST_00004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1


## 컬럼별 데이터 형태가 1개밖에 없는 경우 drop

unique 값이 1개인 컬럼 drop


In [19]:
def remove_single_value_columns(df):
    single_value_cols = []
    for col in df.columns:
        if df[col].nunique() == 1:
            single_value_cols.append(col)

    if single_value_cols:
        print(f"제거할 컬럼 (유니크 값 1개): {single_value_cols}")
        df = df.drop(columns=single_value_cols)
    else:
        print("유니크 값이 1개인 컬럼이 없습니다.")

    return df

In [20]:
customer_train_df = remove_single_value_columns(customer_train_df)
credit_train_df   = remove_single_value_columns(credit_train_df)
sales_train_df    = remove_single_value_columns(sales_train_df)
billing_train_df  = remove_single_value_columns(billing_train_df)
balance_train_df  = remove_single_value_columns(balance_train_df)
channel_train_df  = remove_single_value_columns(channel_train_df)
marketing_train_df= remove_single_value_columns(marketing_train_df)
performance_train_df = remove_single_value_columns(performance_train_df)

제거할 컬럼 (유니크 값 1개): ['이용카드수_체크_가족', '이용금액_R3M_체크_가족', '연회비할인카드수_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M']
제거할 컬럼 (유니크 값 1개): ['시장연체상환여부_R3M']
제거할 컬럼 (유니크 값 1개): ['이용건수_부분무이자_B0M', '이용금액_부분무이자_B0M', '여유_여행이용금액', '납부_렌탈료이용금액', '납부_유선방송이용금액', '납부_건강연금이용금액', '할부건수_부분_3M_R12M', '할부건수_부분_6M_R12M', '할부건수_부분_14M_R12M', '할부금액_부분_3M_R12M', 'RP건수_유선방송_B0M', 'RP건수_건강_B0M', 'RP후경과월_유선방송', 'RP후경과월_건강', '증감_RP건수_유선방송_전월', '증감_RP건수_건강_전월', '이용개월수_당사페이_R6M', '이용금액_당사페이_R6M', '이용금액_당사기타_R6M', '이용건수_당사페이_R6M', '이용건수_당사기타_R6M', '이용금액_당사페이_R3M', '이용금액_당사기타_R3M', '이용건수_당사페이_R3M', '이용건수_당사기타_R3M', '이용금액_당사페이_B0M', '이용금액_당사기타_B0M', '이용건수_당사페이_B0M', '이용건수_당사기타_B0M', '승인거절건수_입력오류_B0M', '승인거절건수_기타_B0M']
제거할 컬럼 (유니크 값 1개): ['대표결제방법코드']
제거할 컬럼 (유니크 값 1개): ['카드론잔액_최종경과월', '최종연체회차', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M', 'RV잔액이월횟수_R3M', '연체잔액_일시불_해외_B0M', '연체잔액_RV일시불_해외_B0M', '연체잔액_할부_해외_B0M', '연체잔액_CA_해외_B0M']
제거할 컬럼 (유니크 값 1개): ['인입횟수_금융_IB_R6M', '인입불만횟수_IB_

In [21]:
customer_test_df     = remove_single_value_columns(customer_test_df)
credit_test_df       = remove_single_value_columns(credit_test_df)
sales_test_df        = remove_single_value_columns(sales_test_df)
billing_test_df      = remove_single_value_columns(billing_test_df)
balance_test_df      = remove_single_value_columns(balance_test_df)
channel_test_df      = remove_single_value_columns(channel_test_df)
marketing_test_df    = remove_single_value_columns(marketing_test_df)
performance_test_df  = remove_single_value_columns(performance_test_df)

제거할 컬럼 (유니크 값 1개): ['이용카드수_체크_가족', '이용금액_R3M_체크_가족', '연회비할인카드수_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M']
제거할 컬럼 (유니크 값 1개): ['시장연체상환여부_R3M']
제거할 컬럼 (유니크 값 1개): ['이용건수_부분무이자_B0M', '이용금액_부분무이자_B0M', '여유_여행이용금액', '납부_렌탈료이용금액', '납부_유선방송이용금액', '납부_건강연금이용금액', '할부건수_무이자_14M_R12M', '할부건수_부분_3M_R12M', '할부건수_부분_6M_R12M', '할부건수_부분_14M_R12M', '할부금액_부분_3M_R12M', '할부금액_부분_6M_R12M', 'RP건수_유선방송_B0M', 'RP건수_건강_B0M', 'RP후경과월_유선방송', 'RP후경과월_건강', '증감_RP건수_유선방송_전월', '증감_RP건수_건강_전월', '이용개월수_당사페이_R6M', '이용금액_당사페이_R6M', '이용금액_당사기타_R6M', '이용건수_당사페이_R6M', '이용건수_당사기타_R6M', '이용금액_당사페이_R3M', '이용금액_당사기타_R3M', '이용건수_당사페이_R3M', '이용건수_당사기타_R3M', '이용금액_당사페이_B0M', '이용금액_당사기타_B0M', '이용건수_당사페이_B0M', '이용건수_당사기타_B0M', '신청건수_ATM_CL_B0', '승인거절건수_입력오류_B0M', '승인거절건수_기타_B0M', '승인거절건수_입력오류_R3M']
제거할 컬럼 (유니크 값 1개): ['대표결제방법코드']
제거할 컬럼 (유니크 값 1개): ['카드론잔액_최종경과월', '연체잔액_대환론_B0M', '최종연체회차', '매각잔액_B1M', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M', 'RV잔액이월횟수_R3M', '연체잔액_일시불_해외_B0M', '

In [22]:
customer_date_columns = ["입회일자_신용", "최종유효년월_신용_이용가능", "최종유효년월_신용_이용", "최종카드발급일자"]
sales_date_columns = ["최종이용일자_기본", "최종이용일자_신판", "최종이용일자_CA", "최종이용일자_카드론", "최종이용일자_체크", "최종이용일자_일시불", "최종이용일자_할부", "최종카드론_대출일자"]

# 입회일자_신용, 최종유효년월_신용_이용가능, 최종유효년월_신용_이용, 최종카드발급일자
#
# 최종이용일자_기본, 최종이용일자_신판, 최종이용일자_CA, 최종이용일자_카드론, 최종이용일자_체크, 최종이용일자_일시불, 최종이용일자_할부, 최종카드론_대출일자
#
#
#
#

## 이상치 처리

IQR 방식으로 이상치 탐색 후 NaN 으로 대치 (날짜 형태의 컬럼은 0으로 처리)

날짜형 데이터의 추가적인 결측치가 있을 경우 0처리

In [23]:
def handle_outliers_with_iqr(df, date_columns=None):
    df_copy = df.copy()

    if date_columns is None:
        date_columns = []

    # 모든 수치형 컬럼 선택
    numeric_cols = df_copy.select_dtypes(include=['int64', 'float64']).columns

    # 각 수치형 컬럼 처리
    for col in numeric_cols:
        # 결측치가 아닌 값으로만 IQR 계산
        valid_data = df_copy[col].dropna()

        if len(valid_data) > 0:  # 유효한 데이터가 있는 경우만 처리
            Q1 = valid_data.quantile(0.25)
            Q3 = valid_data.quantile(0.75)
            IQR = Q3 - Q1

            # 이상치 경계 계산
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # 이상치 찾기
            mask_lower = df_copy[col] < lower_bound
            mask_upper = df_copy[col] > upper_bound

            # 이상치 개수 출력
            outliers_count = mask_lower.sum() + mask_upper.sum()
            if outliers_count > 0:
                print(f"컬럼 '{col}': {outliers_count}개의 이상치 발견 (하한값: {lower_bound}, 상한값: {upper_bound})")

            # 이상치 처리 - date_columns에 포함된 컬럼이면 0으로, 아니면 NaN으로
            if col in date_columns:
                df_copy.loc[mask_lower | mask_upper, col] = 0
                if outliers_count > 0:
                    print(f"  -> 날짜 컬럼으로 간주하여 이상치를 0으로 대체")
            else:
                df_copy.loc[mask_lower | mask_upper, col] = np.nan
                if outliers_count > 0:
                    print(f"  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체")

    return df_copy

In [24]:
customer_train_df   = handle_outliers_with_iqr(customer_train_df, customer_date_columns)
credit_train_df      = handle_outliers_with_iqr(credit_train_df)
sales_train_df      = handle_outliers_with_iqr(sales_train_df, sales_date_columns)
billing_train_df    = handle_outliers_with_iqr(billing_train_df)
balance_train_df    = handle_outliers_with_iqr(balance_train_df)
channel_train_df    = handle_outliers_with_iqr(channel_train_df)
marketing_train_df  = handle_outliers_with_iqr(marketing_train_df)
performance_train_df = handle_outliers_with_iqr(performance_train_df)

컬럼 '회원여부_이용가능': 113820개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '회원여부_이용가능_CA': 287757개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '소지여부_신용': 41887개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '소지카드수_유효_신용': 554051개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '소지카드수_이용가능_신용': 676737개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '입회일자_신용': 153732개의 이상치 발견 (하한값: 19994851.0, 상한값: 20276051.0)
  -> 날짜 컬럼으로 간주하여 이상치를 0으로 대체
컬럼 '입회경과개월수_신용': 127703개의 이상치 발견 (하한값: -113.5, 상한값: 234.5)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '회원여부_연체': 40992개의 이상치 발견 (하한값: 0.0, 상한값: 0.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '이용거절여부_카드론': 474230개의 이상치 발견 (하한값: 0.0, 상한값: 0.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '동의여부_한도증액안내': 236494개의 이상치 발견 (하한값: 0.0, 상한값: 0.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '탈회횟수_발급6개월이내': 97805개의 이상치 발견 (하한값: 0.0, 상한값: 0.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '탈회횟수_발급1년이내': 154282개의 이상치 발견 (하한값: 0

In [25]:
customer_test_df     = handle_outliers_with_iqr(customer_test_df, customer_date_columns)
credit_test_df       = handle_outliers_with_iqr(credit_test_df)
sales_test_df        = handle_outliers_with_iqr(sales_test_df, sales_date_columns)
billing_test_df      = handle_outliers_with_iqr(billing_test_df)
balance_test_df      = handle_outliers_with_iqr(balance_test_df)
channel_test_df      = handle_outliers_with_iqr(channel_test_df)
marketing_test_df    = handle_outliers_with_iqr(marketing_test_df)
performance_test_df  = handle_outliers_with_iqr(performance_test_df)

컬럼 '회원여부_이용가능': 28917개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '회원여부_이용가능_CA': 72748개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '소지여부_신용': 10841개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '소지카드수_유효_신용': 138914개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '소지카드수_이용가능_신용': 169458개의 이상치 발견 (하한값: 1.0, 상한값: 1.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '입회일자_신용': 38178개의 이상치 발견 (하한값: 19994851.0, 상한값: 20276051.0)
  -> 날짜 컬럼으로 간주하여 이상치를 0으로 대체
컬럼 '입회경과개월수_신용': 33377개의 이상치 발견 (하한값: -112.0, 상한값: 232.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '회원여부_연체': 10256개의 이상치 발견 (하한값: 0.0, 상한값: 0.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '이용거절여부_카드론': 119113개의 이상치 발견 (하한값: 0.0, 상한값: 0.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '동의여부_한도증액안내': 59304개의 이상치 발견 (하한값: 0.0, 상한값: 0.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '탈회횟수_발급6개월이내': 24265개의 이상치 발견 (하한값: 0.0, 상한값: 0.0)
  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체
컬럼 '탈회횟수_발급1년이내': 38352개의 이상치 발견 (하한값: 0.0, 상한

In [26]:
def handle_date_columns(df, date_cols):
    for col in date_cols:
        if col in df.columns:
            missing_before = df[col].isnull().sum()
            df[col] = df[col].fillna(0)
            missing_after = df[col].isnull().sum()
            print(f"컬럼 '{col}'의 결측치: {missing_before} -> {missing_after}")
    return df

In [27]:
customer_train_df = handle_date_columns(customer_train_df, customer_date_columns)

sales_train_df = handle_date_columns(sales_train_df, sales_date_columns)

컬럼 '입회일자_신용'의 결측치: 0 -> 0
컬럼 '최종유효년월_신용_이용가능'의 결측치: 210447 -> 0
컬럼 '최종유효년월_신용_이용'의 결측치: 534231 -> 0
컬럼 '최종카드발급일자'의 결측치: 41965 -> 0
컬럼 '최종이용일자_기본'의 결측치: 93072 -> 0
컬럼 '최종이용일자_신판'의 결측치: 93878 -> 0
컬럼 '최종이용일자_CA'의 결측치: 1634513 -> 0
컬럼 '최종이용일자_카드론'의 결측치: 1988330 -> 0
컬럼 '최종이용일자_체크'의 결측치: 1518259 -> 0
컬럼 '최종이용일자_일시불'의 결측치: 98523 -> 0
컬럼 '최종이용일자_할부'의 결측치: 666655 -> 0
컬럼 '최종카드론_대출일자'의 결측치: 1988330 -> 0


In [28]:
customer_test_df = handle_date_columns(customer_test_df, customer_date_columns)

sales_test_df = handle_date_columns(sales_test_df, sales_date_columns)

컬럼 '입회일자_신용'의 결측치: 0 -> 0
컬럼 '최종유효년월_신용_이용가능'의 결측치: 53149 -> 0
컬럼 '최종유효년월_신용_이용'의 결측치: 134817 -> 0
컬럼 '최종카드발급일자'의 결측치: 10854 -> 0
컬럼 '최종이용일자_기본'의 결측치: 23536 -> 0
컬럼 '최종이용일자_신판'의 결측치: 23732 -> 0
컬럼 '최종이용일자_CA'의 결측치: 408920 -> 0
컬럼 '최종이용일자_카드론'의 결측치: 497538 -> 0
컬럼 '최종이용일자_체크'의 결측치: 378741 -> 0
컬럼 '최종이용일자_일시불'의 결측치: 24936 -> 0
컬럼 '최종이용일자_할부'의 결측치: 167573 -> 0
컬럼 '최종카드론_대출일자'의 결측치: 497538 -> 0


## 결측치 처리

각 컬럼별 결측치의 비율에 따라 평균값, 중앙값, 최빈값, drop등으로 처리

In [29]:
def handle_missing_values(df):
    df_copy = df.copy()

    # 결측치가 있는 컬럼 찾기
    columns_with_missing = [col for col in df_copy.columns if df_copy[col].isnull().sum() > 0]

    for col in columns_with_missing:
        missing_count = df_copy[col].isnull().sum()
        missing_ratio = missing_count / len(df_copy)
        print(f"컬럼 '{col}'의 결측치: {missing_count}개 ({missing_ratio:.4f})")

        # 결측 비율에 따른 처리
        if missing_ratio > 0.75:
            # 75% 초과 결측: 컬럼 제거 고려
            print(f"  -> 결측치 비율이 75%를 초과합니다. 해당 컬럼 제거")
            df_copy = df_copy.drop(columns=[col])

        elif missing_ratio > 0.5:
            # 50~75% 결측: 컬럼 유지하되 결측치는 분포의 중심값으로 대체
            if df_copy[col].dtype in ['int64', 'float64']:
                # 수치형: 중앙값(median)으로 대체
                median_val = df_copy[col].median()
                df_copy[col] = df_copy[col].fillna(median_val)
                print(f"  -> 결측 비율이 높은 수치형 컬럼이므로 중앙값 {median_val}으로 대체")
            else:
                # 범주형: 최빈값(mode)으로 대체
                mode_val = df_copy[col].mode()[0] if not df_copy[col].mode().empty else "unknown"
                df_copy[col] = df_copy[col].fillna(mode_val)
                print(f"  -> 결측 비율이 높은 범주형 컬럼이므로 최빈값 '{mode_val}'으로 대체")

        elif missing_ratio > 0.3:
            # 30~50% 결측: 수치형은 평균/분포 기반, 범주형은 최빈값 대체
            if df_copy[col].dtype in ['int64', 'float64']:
                # 왜도(skewness) 확인하여 처리
                skew = df_copy[col].skew()
                if abs(skew) > 1:  # 비대칭 분포인 경우
                    median_val = df_copy[col].median()
                    df_copy[col] = df_copy[col].fillna(median_val)
                    print(f"  -> 분포가 비대칭(skew={skew:.2f})이므로 중앙값 {median_val}으로 대체")
                else:  # 대칭 분포인 경우
                    mean_val = df_copy[col].mean()
                    df_copy[col] = df_copy[col].fillna(mean_val)
                    print(f"  -> 분포가 대칭(skew={skew:.2f})이므로 평균값 {mean_val}으로 대체")
            else:
                # 범주형: 최빈값으로 대체
                mode_val = df_copy[col].mode()[0] if not df_copy[col].mode().empty else "unknown"
                df_copy[col] = df_copy[col].fillna(mode_val)
                print(f"  -> 범주형 컬럼이므로 최빈값 '{mode_val}'으로 대체")

        else:
            # 30% 이하 결측: 일반적인 대체 방법 적용
            if df_copy[col].dtype in ['int64', 'float64']:
                # 왜도 확인
                skew = df_copy[col].skew()
                if abs(skew) > 1:
                    median_val = df_copy[col].median()
                    df_copy[col] = df_copy[col].fillna(median_val)
                    print(f"  -> 분포가 비대칭(skew={skew:.2f})이므로 중앙값 {median_val}으로 대체")
                else:
                    mean_val = df_copy[col].mean()
                    df_copy[col] = df_copy[col].fillna(mean_val)
                    print(f"  -> 분포가 대칭(skew={skew:.2f})이므로 평균값 {mean_val}으로 대체")
            else:
                mode_val = df_copy[col].mode()[0] if not df_copy[col].mode().empty else "unknown"
                df_copy[col] = df_copy[col].fillna(mode_val)
                print(f"  -> 범주형 컬럼이므로 최빈값 '{mode_val}'으로 대체")

    return df_copy

In [30]:
customer_train_df   = handle_missing_values(customer_train_df)
credit_train_df      = handle_missing_values(credit_train_df)
sales_train_df      = handle_missing_values(sales_train_df)
billing_train_df    = handle_missing_values(billing_train_df)
balance_train_df    = handle_missing_values(balance_train_df)
channel_train_df    = handle_missing_values(channel_train_df)
marketing_train_df  = handle_missing_values(marketing_train_df)
performance_train_df = handle_missing_values(performance_train_df)

컬럼 '회원여부_이용가능'의 결측치: 113820개 (0.0474)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '회원여부_이용가능_CA'의 결측치: 287757개 (0.1199)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '소지여부_신용'의 결측치: 41887개 (0.0175)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '소지카드수_유효_신용'의 결측치: 554051개 (0.2309)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '소지카드수_이용가능_신용'의 결측치: 676737개 (0.2820)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '입회경과개월수_신용'의 결측치: 127703개 (0.0532)
  -> 분포가 비대칭(skew=1.14)이므로 중앙값 47.0으로 대체
컬럼 '회원여부_연체'의 결측치: 40992개 (0.0171)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '이용거절여부_카드론'의 결측치: 474230개 (0.1976)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '동의여부_한도증액안내'의 결측치: 236494개 (0.0985)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '가입통신회사코드'의 결측치: 387570개 (0.1615)
  -> 범주형 컬럼이므로 최빈값 'S사'으로 대체
컬럼 '탈회횟수_발급6개월이내'의 결측치: 97805개 (0.0408)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '탈회횟수_발급1년이내'의 결측치: 154282개 (0.0643)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '직장시도명'의 결측치: 244969개 (0.1021)
  -> 범주형 컬럼이므로 최빈값 '경기'으로 대체
컬럼 '마케팅동의여부

In [31]:
customer_test_df     = handle_missing_values(customer_test_df)
credit_test_df       = handle_missing_values(credit_test_df)
sales_test_df        = handle_missing_values(sales_test_df)
billing_test_df      = handle_missing_values(billing_test_df)
balance_test_df      = handle_missing_values(balance_test_df)
channel_test_df      = handle_missing_values(channel_test_df)
marketing_test_df    = handle_missing_values(marketing_test_df)
performance_test_df  = handle_missing_values(performance_test_df)

컬럼 '회원여부_이용가능'의 결측치: 28917개 (0.0482)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '회원여부_이용가능_CA'의 결측치: 72748개 (0.1212)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '소지여부_신용'의 결측치: 10841개 (0.0181)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '소지카드수_유효_신용'의 결측치: 138914개 (0.2315)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '소지카드수_이용가능_신용'의 결측치: 169458개 (0.2824)
  -> 분포가 대칭(skew=0.00)이므로 평균값 1.0으로 대체
컬럼 '입회경과개월수_신용'의 결측치: 33377개 (0.0556)
  -> 분포가 비대칭(skew=1.13)이므로 중앙값 46.0으로 대체
컬럼 '회원여부_연체'의 결측치: 10256개 (0.0171)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '이용거절여부_카드론'의 결측치: 119113개 (0.1985)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '동의여부_한도증액안내'의 결측치: 59304개 (0.0988)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '가입통신회사코드'의 결측치: 97083개 (0.1618)
  -> 범주형 컬럼이므로 최빈값 'S사'으로 대체
컬럼 '탈회횟수_발급6개월이내'의 결측치: 24265개 (0.0404)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '탈회횟수_발급1년이내'의 결측치: 38352개 (0.0639)
  -> 분포가 대칭(skew=0.00)이므로 평균값 0.0으로 대체
컬럼 '직장시도명'의 결측치: 62233개 (0.1037)
  -> 범주형 컬럼이므로 최빈값 '경기'으로 대체
컬럼 '마케팅동의여부'의 결측치:

# 라벨 인코딩 처리

In [32]:
from sklearn.preprocessing import LabelEncoder

In [33]:
def label_encode_dataframe(df, exclude_cols=None):
    # 원본 데이터프레임을 수정하지 않기 위해 복사
    df_encoded = df.copy()

    # 제외할 컬럼 목록 설정
    if exclude_cols is None:
        exclude_cols = []

    # object 타입 컬럼 찾기
    object_columns = df_encoded.select_dtypes(include=['object']).columns

    # 인코딩할 컬럼 (제외 컬럼 제외)
    encode_columns = [col for col in object_columns if col not in exclude_cols]

    if len(encode_columns) == 0:
        print("인코딩할 object 타입 컬럼이 없습니다.")
        return df_encoded

    print(f"총 {len(encode_columns)}개의 object 타입 컬럼에 라벨 인코딩을 적용합니다.")

    # 각 컬럼에 라벨 인코딩 적용
    for col in encode_columns:
        print(f"컬럼 '{col}' 인코딩 중...")

        # 해당 컬럼의 유니크 값 수 확인
        unique_count = df_encoded[col].nunique()
        print(f"  -> 유니크 값 수: {unique_count}")

        # 누락된 값이 있는지 확인
        null_count = df_encoded[col].isnull().sum()
        if null_count > 0:
            print(f"  -> 주의: {null_count}개의 결측치가 있습니다.")

        # 라벨 인코더 생성 및 적용
        le = LabelEncoder()

        # 결측치가 있는 경우 임시로 처리 (인코딩 후 다시 NaN으로 변경)
        temp_col = df_encoded[col].copy()
        mask_null = temp_col.isnull()

        if mask_null.any():
            # 결측치는 임시로 특수 문자열로 대체
            temp_col = temp_col.fillna('NULL_VALUE_FOR_ENCODING')

        # 인코딩 적용
        temp_col = le.fit_transform(temp_col.astype(str))

        # 인코딩된 값 데이터프레임에 대입
        df_encoded[col] = temp_col

        # 결측치가 있었다면 다시 NaN으로 변경
        if mask_null.any():
            df_encoded.loc[mask_null, col] = np.nan

        print(f"  -> 인코딩 완료: {len(le.classes_)}개의 클래스로 변환")

    print("모든 컬럼 인코딩 완료!")
    return df_encoded  # 인코딩된 데이터프레임만 반환

In [34]:
exclude_columns = ['ID', 'Segment']

customer_train_df   = label_encode_dataframe(customer_train_df, exclude_columns)
credit_train_df = label_encode_dataframe(credit_train_df, exclude_columns)
sales_train_df = label_encode_dataframe(sales_train_df, exclude_columns)
billing_train_df = label_encode_dataframe(billing_train_df, exclude_columns)
channel_train_df = label_encode_dataframe(channel_train_df, exclude_columns)
marketing_train_df = label_encode_dataframe(marketing_train_df, exclude_columns)
performance_train_df = label_encode_dataframe(performance_train_df, exclude_columns)

총 8개의 object 타입 컬럼에 라벨 인코딩을 적용합니다.
컬럼 '연령' 인코딩 중...
  -> 유니크 값 수: 6
  -> 인코딩 완료: 6개의 클래스로 변환
컬럼 '가입통신회사코드' 인코딩 중...
  -> 유니크 값 수: 3
  -> 인코딩 완료: 3개의 클래스로 변환
컬럼 '거주시도명' 인코딩 중...
  -> 유니크 값 수: 17
  -> 인코딩 완료: 17개의 클래스로 변환
컬럼 '직장시도명' 인코딩 중...
  -> 유니크 값 수: 17
  -> 인코딩 완료: 17개의 클래스로 변환
컬럼 '_1순위신용체크구분' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 '_2순위신용체크구분' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 '연회비발생카드수_B0M' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 'Life_Stage' 인코딩 중...
  -> 유니크 값 수: 7
  -> 인코딩 완료: 7개의 클래스로 변환
모든 컬럼 인코딩 완료!
총 5개의 object 타입 컬럼에 라벨 인코딩을 적용합니다.
컬럼 '자발한도감액횟수_R12M' 인코딩 중...
  -> 유니크 값 수: 3
  -> 인코딩 완료: 3개의 클래스로 변환
컬럼 '한도증액횟수_R12M' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 '카드론동의여부' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 'RV전환가능여부' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 '한도심사요청건수' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
모든 컬럼 인코딩 완료!
총 10개의 object 타입 컬럼에 라벨 인코딩을 적용합니다.
컬럼 '_1순위업종' 인코딩 중...
  ->

In [35]:
exclude_columns = ['ID']

customer_test_df   = label_encode_dataframe(customer_test_df, exclude_columns)
credit_test_df     = label_encode_dataframe(credit_test_df, exclude_columns)
sales_test_df      = label_encode_dataframe(sales_test_df, exclude_columns)
billing_test_df    = label_encode_dataframe(billing_test_df, exclude_columns)
channel_test_df    = label_encode_dataframe(channel_test_df, exclude_columns)
marketing_test_df  = label_encode_dataframe(marketing_test_df, exclude_columns)
performance_test_df= label_encode_dataframe(performance_test_df, exclude_columns)

총 8개의 object 타입 컬럼에 라벨 인코딩을 적용합니다.
컬럼 '연령' 인코딩 중...
  -> 유니크 값 수: 6
  -> 인코딩 완료: 6개의 클래스로 변환
컬럼 '가입통신회사코드' 인코딩 중...
  -> 유니크 값 수: 3
  -> 인코딩 완료: 3개의 클래스로 변환
컬럼 '거주시도명' 인코딩 중...
  -> 유니크 값 수: 17
  -> 인코딩 완료: 17개의 클래스로 변환
컬럼 '직장시도명' 인코딩 중...
  -> 유니크 값 수: 17
  -> 인코딩 완료: 17개의 클래스로 변환
컬럼 '_1순위신용체크구분' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 '_2순위신용체크구분' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 '연회비발생카드수_B0M' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 'Life_Stage' 인코딩 중...
  -> 유니크 값 수: 7
  -> 인코딩 완료: 7개의 클래스로 변환
모든 컬럼 인코딩 완료!
총 5개의 object 타입 컬럼에 라벨 인코딩을 적용합니다.
컬럼 '자발한도감액횟수_R12M' 인코딩 중...
  -> 유니크 값 수: 3
  -> 인코딩 완료: 3개의 클래스로 변환
컬럼 '한도증액횟수_R12M' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 '카드론동의여부' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 'RV전환가능여부' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
컬럼 '한도심사요청건수' 인코딩 중...
  -> 유니크 값 수: 2
  -> 인코딩 완료: 2개의 클래스로 변환
모든 컬럼 인코딩 완료!
총 10개의 object 타입 컬럼에 라벨 인코딩을 적용합니다.
컬럼 '_1순위업종' 인코딩 중...
  ->

In [36]:
customer_test_df

,기준년월,ID,남녀구분코드,연령,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,...,이용가능여부_해외겸용_신용_본인,이용여부_3M_해외겸용_신용_본인,연회비발생카드수_B0M,기본연회비_B0M,제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807.0,TEST_00000,1.0,2,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,4,45.0
1,201807.0,TEST_00001,1.0,4,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,3,19.0
2,201807.0,TEST_00002,1.0,2,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,4,5.0
3,201807.0,TEST_00003,2.0,2,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,4,7.0
4,201807.0,TEST_00004,2.0,2,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812.0,TEST_99995,2.0,4,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0.0
599996,201812.0,TEST_99996,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,6,12.0
599997,201812.0,TEST_99997,2.0,1,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,4,11.0
599998,201812.0,TEST_99998,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0,1.0


In [37]:
# 기본 데이터프레임 설정
merged_train_df = customer_train_df.copy()

# 나머지 데이터프레임들을 병합
for df_name in ['credit_train_df', 'sales_train_df', 'billing_train_df',
                'channel_train_df', 'marketing_train_df', 'performance_train_df']:
    print(f"{df_name} 병합 중...")
    # '기준년월'과 'ID' 모두를 기준으로 병합
    merged_train_df = merged_train_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"현재 병합된 데이터프레임 크기: {merged_train_df.shape}")
    del globals()[df_name]
    gc.collect()

credit_train_df 병합 중...
현재 병합된 데이터프레임 크기: (2400000, 106)
sales_train_df 병합 중...
현재 병합된 데이터프레임 크기: (2400000, 469)
billing_train_df 병합 중...
현재 병합된 데이터프레임 크기: (2400000, 512)
channel_train_df 병합 중...
현재 병합된 데이터프레임 크기: (2400000, 579)
marketing_train_df 병합 중...
현재 병합된 데이터프레임 크기: (2400000, 619)
performance_train_df 병합 중...
현재 병합된 데이터프레임 크기: (2400000, 666)


In [38]:
merged_test_df = customer_test_df.copy()

# 나머지 데이터프레임들을 병합
for df_name in ['credit_test_df', 'sales_test_df', 'billing_test_df',
                'channel_test_df', 'marketing_test_df', 'performance_test_df']:
    print(f"{df_name} 병합 중...")
    # '기준년월'과 'ID' 모두를 기준으로 병합
    merged_test_df = merged_test_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"현재 병합된 데이터프레임 크기: {merged_test_df.shape}")
    del globals()[df_name]
    gc.collect()

credit_test_df 병합 중...
현재 병합된 데이터프레임 크기: (600000, 105)
sales_test_df 병합 중...
현재 병합된 데이터프레임 크기: (600000, 464)
billing_test_df 병합 중...
현재 병합된 데이터프레임 크기: (600000, 507)
channel_test_df 병합 중...
현재 병합된 데이터프레임 크기: (600000, 574)
marketing_test_df 병합 중...
현재 병합된 데이터프레임 크기: (600000, 613)
performance_test_df 병합 중...
현재 병합된 데이터프레임 크기: (600000, 660)


In [39]:
import xgboost as xgb

In [40]:
feature_cols = [col for col in merged_train_df.columns if col not in ["ID", "Segment"]]

X = merged_train_df[feature_cols].copy()
y = merged_train_df["Segment"].copy()

le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

# 학습

In [42]:
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

model.fit(X, y_encoded)

KeyboardInterrupt: 